In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('datafiles/june4/data1.csv', sep=',')

In [4]:
df['ratio']=df['Integral 1200']/df['Integral 7400']

In [5]:
df

,Time,Integral 1200,Integral 7400,Unnamed: 3,ratio
0,1.528150e+09,67986,111746.0,NaN,0.608398
1,1.528150e+09,37655,53317.0,NaN,0.706248
2,1.528150e+09,70037,117233.0,NaN,0.597417
3,1.528150e+09,53668,88457.0,NaN,0.606713
4,1.528150e+09,267520,448843.0,NaN,0.596021
5,1.528150e+09,41360,68061.0,NaN,0.607690
6,1.528150e+09,40625,66594.0,NaN,0.610040
7,1.528150e+09,36690,58739.0,NaN,0.624628
8,1.528150e+09,72417,122073.0,NaN,0.593227
9,1.528150e+09,52161,86373.0,NaN,0.603904
